In [2]:
import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.manifold import TSNE
import pickle
# from Bio.Cluster import kcluster
import os
import numpy as np
import yaml
import os
import pandas
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
import seaborn as sns
# import pyclustering
import matplotlib.pylab as plt
import seaborn as sb
from scipy.spatial.distance import cdist

from shutil import copyfile
import matplotlib as plt
# ------- SERVER EXTENSIONS ---------
lib =  r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities/droplet_dataset'
lib2 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities'
lib3 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/data_analysis'
lib4 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy'
lib5 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/scripts'
import sys
import pandas as pd, os
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
# ------- SERVER EXTENSIONS ---------

# Creating FASTQ files
* 2 samples each time. you should only specify their names and the path for the joint CSV.
* Should be a folder with CSV mapping the barcodes for both samples.
* The output is a bash file which you can run and create the complete FASTQ files.

In [3]:
path = r'/storage/md_keren/shitay/10X/M157_M158/M157_M158_10-7-20.csv' ## csv path

sample1_name = 'M157'
sample2_name = 'M158'

output_bash_file = fr'/storage/md_keren/shitay/10X/M157_M158/pkgs/SN0219717/combine_fastq_{sample1_name}_{sample2_name}.sh' ## the path of the created .sh file
FASTQ_output_path = r'/storage/md_keren/shitay/10X/fastq' ## the output path of the final FASTQ file after you run the .sh file that build the fastq


# It is a unique prefix that each of the row files contains. 
# look over the folder '/storage/md_keren/shitay/10X/M149_M150/pkgs/SN0217504/',
# the prefix there for example is '_HVM2VBGXG.' 
# prefix = '_HVM2VBGXG.' 
prefix = '_HVWLTBGXH.'

In [4]:
df = pd.read_csv(path)[['SampleName', 'IndexBarcode1']]
values = df.values 
sample1_values = [[s[0],s[1]] for s in values if sample1_name in s[0]]
sample1_values = {int(barcode[0].split('_')[1]): barcode[1] for barcode in sample1_values}
sample2_values = [[s[0],s[1]] for s in values if sample2_name in s[0]]
sample2_values = {int(barcode[0].split('_')[1]): barcode[1] for barcode in sample2_values}


y = [[1, 2],[3, 4]]
[z for x in y for z in x]
sample1_values

{1: 'ATAGTTAC', 2: 'TGCTGAGT', 3: 'CCTACGTA', 4: 'GAGCACCG'}

In [5]:

def synthesize_command_for_sample(barcode_values, sample_name):
    n_of_channels = 4
    n_of_directions = 2
    n_of_barcodes = len(barcode_values)
    lines = []
    lines.append(f'mkdir {FASTQ_output_path}/{sample_name}')
    for direction in range(1, 3):
        line = 'cat '
        for channel in range(1, n_of_channels+1):
            for barcode_idx in range(1, n_of_barcodes+1):
                barcode = barcode_values[barcode_idx]
                line += str(channel)+prefix+str(channel)+'.'+barcode+'.unmapped.'+str(direction)+ '.fastq.gz '
        line += f'> {FASTQ_output_path}/{sample_name}/{sample_name}_S1_L001_R{direction}_001.fastq.gz'
        lines.append(line)      
    return lines
res = synthesize_command_for_sample(sample1_values, sample1_name) + synthesize_command_for_sample(sample2_values, sample2_name)
res

['mkdir /storage/md_keren/shitay/10X/fastq/M157',
 'cat 1_HVWLTBGXH.1.ATAGTTAC.unmapped.1.fastq.gz 1_HVWLTBGXH.1.TGCTGAGT.unmapped.1.fastq.gz 1_HVWLTBGXH.1.CCTACGTA.unmapped.1.fastq.gz 1_HVWLTBGXH.1.GAGCACCG.unmapped.1.fastq.gz 2_HVWLTBGXH.2.ATAGTTAC.unmapped.1.fastq.gz 2_HVWLTBGXH.2.TGCTGAGT.unmapped.1.fastq.gz 2_HVWLTBGXH.2.CCTACGTA.unmapped.1.fastq.gz 2_HVWLTBGXH.2.GAGCACCG.unmapped.1.fastq.gz 3_HVWLTBGXH.3.ATAGTTAC.unmapped.1.fastq.gz 3_HVWLTBGXH.3.TGCTGAGT.unmapped.1.fastq.gz 3_HVWLTBGXH.3.CCTACGTA.unmapped.1.fastq.gz 3_HVWLTBGXH.3.GAGCACCG.unmapped.1.fastq.gz 4_HVWLTBGXH.4.ATAGTTAC.unmapped.1.fastq.gz 4_HVWLTBGXH.4.TGCTGAGT.unmapped.1.fastq.gz 4_HVWLTBGXH.4.CCTACGTA.unmapped.1.fastq.gz 4_HVWLTBGXH.4.GAGCACCG.unmapped.1.fastq.gz > /storage/md_keren/shitay/10X/fastq/M157/M157_S1_L001_R1_001.fastq.gz',
 'cat 1_HVWLTBGXH.1.ATAGTTAC.unmapped.2.fastq.gz 1_HVWLTBGXH.1.TGCTGAGT.unmapped.2.fastq.gz 1_HVWLTBGXH.1.CCTACGTA.unmapped.2.fastq.gz 1_HVWLTBGXH.1.GAGCACCG.unmapped.2.fastq.gz 2_HVW

In [6]:
# output_bash_file
with open(output_bash_file, "w") as text_file:
    for line in res:
        text_file.write(line)
        text_file.write('\n')